In [2]:
import os
import re
import json
import sqlite3

import pandas as pd
import numpy as np
from glob import glob

import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', 50)

In [3]:
os.chdir('../../')

In [4]:
result_paths = glob('./output/*/*/*.csv')

In [5]:
results = []
for p in result_paths:
    df = pd.read_csv(p, index_col=0)
    result = dict(df.apply(lambda x: np.nanmean(x)))
    configs = p.split('/')[2]
    is_imbalance, split_strategy, resampling_strategy, \
    do_copypaste, paste_by = configs.replace('skf_prop','skfprop').split('_')
    mIoU = np.nanmean(list(result.values()))
    random_state= p.split('/')[3]
    result.update(
        configs = configs,
        is_imbalance = is_imbalance,
        split_strategy = split_strategy,
        resampling_strategy = resampling_strategy,
        do_copypaste = do_copypaste,
        paste_by = paste_by,
        random_state = random_state,
        test_mIoU = mIoU
    )
    results.append(result)
    

In [6]:
result_df = pd.DataFrame(results)
result_df["is_imbalance"] = result_df.is_imbalance == "TRUE"
result_df["do_copypaste"] = result_df.do_copypaste == "TRUE"

label_cols = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat',
'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person',
'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor', 'background']

search_spaces = ['is_imbalance', 'split_strategy', 'resampling_strategy', 'do_copypaste', 'paste_by']

non_label_cols = [col for col in result_df.columns if col not in label_cols]
result_df = result_df[non_label_cols + label_cols]



In [7]:
pd.DataFrame(result_df.groupby(search_spaces)[['test_mIoU']+label_cols].mean()).reset_index()

,is_imbalance,split_strategy,resampling_strategy,do_copypaste,paste_by,test_mIoU,aeroplane,bicycle,bird,boat,bottle,bus,car,cat,chair,cow,diningtable,dog,horse,motorbike,person,pottedplant,sheep,sofa,train,tvmonitor,background
0,False,mskf,x2,False,none,0.555109,0.654149,0.282342,0.646530,0.521285,0.360246,0.703331,0.476271,0.751790,0.180076,0.714085,0.461101,0.683473,0.637727,0.649251,0.566585,0.218413,0.632647,0.444444,0.717082,0.476606,0.879851
1,False,random,x2,False,none,0.555764,0.658487,0.285800,0.641149,0.521367,0.362211,0.719109,0.469919,0.759668,0.180420,0.717171,0.459377,0.689982,0.627426,0.644140,0.566040,0.229399,0.616648,0.426433,0.734207,0.481672,0.880416
2,True,mskf,cnt,False,none,0.518855,0.601401,0.238893,0.534862,0.508663,0.252073,0.710929,0.449996,0.662532,0.058378,0.694770,0.396763,0.692225,0.629112,0.643607,0.541202,0.234817,0.592788,0.412543,0.715874,0.453829,0.870707
3,True,mskf,cnt,True,cnt,0.528834,0.619071,0.278994,0.552960,0.492692,0.286117,0.711082,0.458033,0.682652,0.093502,0.664231,0.412092,0.685738,0.627890,0.622289,0.543527,0.269585,0.619472,0.435119,0.711396,0.470887,0.868178
4,True,mskf,cnt,True,performance,0.529656,0.605054,0.279017,0.510021,0.526471,0.304825,0.720491,0.462681,0.654150,0.100524,0.692089,0.366220,0.689769,0.608551,0.631195,0.547929,0.294601,0.614052,0.445312,0.714417,0.486378,0.869037
5,True,mskf,x2,False,none,0.517678,0.572409,0.227446,0.509173,0.511640,0.259387,0.718869,0.446501,0.666308,0.049363,0.683673,0.410245,0.682118,0.632865,0.645259,0.548012,0.234347,0.612732,0.412830,0.729565,0.450262,0.868233
6,True,random,x2,False,none,0.518869,0.545740,0.212530,0.525782,0.511430,0.274431,0.710647,0.471677,0.663291,0.049116,0.704459,0.386190,0.704280,0.633196,0.650438,0.553209,0.245455,0.602148,0.402915,0.711910,0.467336,0.870059
7,True,random,x2,True,cnt,0.531912,0.601056,0.256784,0.554669,0.502715,0.333971,0.724619,0.467937,0.673928,0.104954,0.679059,0.394572,0.681950,0.625286,0.630751,0.554687,0.269211,0.619132,0.434056,0.725580,0.464499,0.870729
8,True,random,x2,True,performance,0.531783,0.548969,0.253544,0.533152,0.529475,0.326867,0.726580,0.476222,0.653660,0.097984,0.676347,0.389443,0.697141,0.647493,0.645632,0.556976,0.290034,0.617863,0.430945,0.728808,0.471367,0.868934
9,True,skfprop,cluster,False,none,0.510055,0.528647,0.196652,0.478498,0.498604,0.260868,0.708872,0.435136,0.656348,0.050411,0.687986,0.389929,0.698500,0.637502,0.645161,0.546821,0.238495,0.601619,0.416707,0.709685,0.455140,0.869583


In [8]:
imbalanced_labels =  "bird, cat, aeroplane, bicycle, bottle, chair".split(', ')

balaced_labels = list(set(label_cols) - set(imbalanced_labels))

In [9]:
pd.DataFrame(result_df.groupby(search_spaces)[['test_mIoU']+imbalanced_labels].mean()).reset_index()

,is_imbalance,split_strategy,resampling_strategy,do_copypaste,paste_by,test_mIoU,bird,cat,aeroplane,bicycle,bottle,chair
0,False,mskf,x2,False,none,0.555109,0.646530,0.751790,0.654149,0.282342,0.360246,0.180076
1,False,random,x2,False,none,0.555764,0.641149,0.759668,0.658487,0.285800,0.362211,0.180420
2,True,mskf,cnt,False,none,0.518855,0.534862,0.662532,0.601401,0.238893,0.252073,0.058378
3,True,mskf,cnt,True,cnt,0.528834,0.552960,0.682652,0.619071,0.278994,0.286117,0.093502
4,True,mskf,cnt,True,performance,0.529656,0.510021,0.654150,0.605054,0.279017,0.304825,0.100524
5,True,mskf,x2,False,none,0.517678,0.509173,0.666308,0.572409,0.227446,0.259387,0.049363
6,True,random,x2,False,none,0.518869,0.525782,0.663291,0.545740,0.212530,0.274431,0.049116
7,True,random,x2,True,cnt,0.531912,0.554669,0.673928,0.601056,0.256784,0.333971,0.104954
8,True,random,x2,True,performance,0.531783,0.533152,0.653660,0.548969,0.253544,0.326867,0.097984
9,True,skfprop,cluster,False,none,0.510055,0.478498,0.656348,0.528647,0.196652,0.260868,0.050411


In [10]:
pd.DataFrame(result_df.groupby(search_spaces)[['test_mIoU']+balaced_labels].mean()).reset_index()

,is_imbalance,split_strategy,resampling_strategy,do_copypaste,paste_by,test_mIoU,person,cow,dog,bus,train,tvmonitor,motorbike,background,pottedplant,diningtable,horse,boat,sheep,car,sofa
0,False,mskf,x2,False,none,0.555109,0.566585,0.714085,0.683473,0.703331,0.717082,0.476606,0.649251,0.879851,0.218413,0.461101,0.637727,0.521285,0.632647,0.476271,0.444444
1,False,random,x2,False,none,0.555764,0.566040,0.717171,0.689982,0.719109,0.734207,0.481672,0.644140,0.880416,0.229399,0.459377,0.627426,0.521367,0.616648,0.469919,0.426433
2,True,mskf,cnt,False,none,0.518855,0.541202,0.694770,0.692225,0.710929,0.715874,0.453829,0.643607,0.870707,0.234817,0.396763,0.629112,0.508663,0.592788,0.449996,0.412543
3,True,mskf,cnt,True,cnt,0.528834,0.543527,0.664231,0.685738,0.711082,0.711396,0.470887,0.622289,0.868178,0.269585,0.412092,0.627890,0.492692,0.619472,0.458033,0.435119
4,True,mskf,cnt,True,performance,0.529656,0.547929,0.692089,0.689769,0.720491,0.714417,0.486378,0.631195,0.869037,0.294601,0.366220,0.608551,0.526471,0.614052,0.462681,0.445312
5,True,mskf,x2,False,none,0.517678,0.548012,0.683673,0.682118,0.718869,0.729565,0.450262,0.645259,0.868233,0.234347,0.410245,0.632865,0.511640,0.612732,0.446501,0.412830
6,True,random,x2,False,none,0.518869,0.553209,0.704459,0.704280,0.710647,0.711910,0.467336,0.650438,0.870059,0.245455,0.386190,0.633196,0.511430,0.602148,0.471677,0.402915
7,True,random,x2,True,cnt,0.531912,0.554687,0.679059,0.681950,0.724619,0.725580,0.464499,0.630751,0.870729,0.269211,0.394572,0.625286,0.502715,0.619132,0.467937,0.434056
8,True,random,x2,True,performance,0.531783,0.556976,0.676347,0.697141,0.726580,0.728808,0.471367,0.645632,0.868934,0.290034,0.389443,0.647493,0.529475,0.617863,0.476222,0.430945
9,True,skfprop,cluster,False,none,0.510055,0.546821,0.687986,0.698500,0.708872,0.709685,0.455140,0.645161,0.869583,0.238495,0.389929,0.637502,0.498604,0.601619,0.435136,0.416707


In [11]:
train_logs = open('/home/nys/nni-experiments/08bsa6re/log/nnimanager.log')
train_logs = train_logs.read().splitlines()
start_times = [log for log in train_logs if "WAITING to RUNNING" in log]
end_times = [log for log in train_logs if "RUNNING to SUCCEEDED" in log]
configs = [log for log in train_logs if "dispatcher: TR" in log]

In [12]:
assert len(start_times) == len(end_times)
assert len(start_times) == len(configs)

In [13]:
box_bracket = r"\[.*?\]"
param_str = "\"parameters\": "
curly_bracket = param_str + "\{.*?\}"
job_name = "Trial job \w+"

config_times = [re.findall(box_bracket, log)[0] for log in configs]
config_args = [json.loads(re.sub(param_str, "",
                          re.findall(curly_bracket, log)[0])) for log in configs]

starts = [re.findall(box_bracket, log)[0] for log in start_times]
ends = [re.findall(box_bracket, log)[0] for log in end_times]
job_names = [re.sub("Trial job ","",re.findall(job_name, log)[0]) for log in start_times]


In [14]:
log_df = pd.DataFrame(config_args)

In [15]:

log_df['job_name'] = job_names
log_df['config_time'] = config_times
log_df['start_time'] = starts
log_df['end_time'] = ends
log_df['config_time'] = config_times

time_cols = [col for col in log_df.columns if 'time' in col]

for col in (time_cols):
    log_df[col] = log_df[col].apply(lambda x: x.replace("[", "").replace("]",""))
    log_df[col] = pd.to_datetime(log_df[col])

log_df['train_time'] = log_df['end_time'] - log_df['start_time']



In [16]:
con = sqlite3.connect('/home/nys/nni-experiments/08bsa6re/db/nni.sqlite')
cur = con.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

print(cur.fetchall())

exp_profile = pd.read_sql_query("SELECT * FROM ExperimentProfile;",con)
metric_data = pd.read_sql_query("SELECT * FROM MetricData;",con)
trial_event = pd.read_sql_query("SELECT * FROM TrialJobEvent;",con)

display(exp_profile.head(3))
display(metric_data.head(3))
display(trial_event.head(3))

[('TrialJobEvent',), ('MetricData',), ('ExperimentProfile',)]


,params,id,execDuration,startTime,endTime,logDir,nextSequenceId,revision
0,"{""searchSpaceFile"":""/home/nys/Desktop/study_20...",08bsa6re,0,1655635275953,NaN,/home/nys/nni-experiments/08bsa6re,0,1
1,"{""searchSpaceFile"":""/home/nys/Desktop/study_20...",08bsa6re,0,1655635275953,NaN,/home/nys/nni-experiments/08bsa6re,0,2
2,"{""searchSpaceFile"":""/home/nys/Desktop/study_20...",08bsa6re,5,1655635275953,NaN,/home/nys/nni-experiments/08bsa6re,1,3


,timestamp,trialJobId,parameterId,type,sequence,data
0,1655635577605,jYQpo,0,PERIODICAL,0,"""0.5529635354201905"""
1,1655635863536,jYQpo,0,PERIODICAL,1,"""0.5696392811953479"""
2,1655636149676,jYQpo,0,PERIODICAL,2,"""0.5905676542029943"""


,timestamp,trialJobId,event,data,logPath,sequenceId,message
0,1655635281050,jYQpo,WAITING,"{""parameter_id"": 0, ""parameter_source"": ""algor...",file://localhost:/home/nys/nni-experiments/08b...,0,None
1,1655635286073,jYQpo,RUNNING,None,file://localhost:/home/nys/nni-experiments/08b...,0,None
2,1655657098615,jYQpo,SUCCEEDED,None,file://localhost:/home/nys/nni-experiments/08b...,0,None


In [17]:

metric_data["epoch_val_mIoU"] = metric_data.data.str.replace("\"", "").astype(float)
max_epoch = metric_data.groupby('trialJobId')['sequence', 'epoch_val_mIoU'].max()
max_epoch = max_epoch.reset_index()
max_epoch.columns = ["job_name", 'train_epoch', "epoch_val_mIoU"]


In [18]:
log_df = pd.merge(log_df, max_epoch, on="job_name", how="left")


In [19]:


log_df['avg_epoch_time'] = log_df.train_time / log_df.train_epoch

log_df['configs'] = log_df.configs.str.replace(" ","_")
log_df = log_df.rename(columns={"split_rs": "random_state"})
log_df['random_state']= log_df.random_state.astype(str)
result_df['random_state']= result_df.random_state.astype(str)

display(log_df.head(3))

,random_state,split_fnum,configs,job_name,config_time,start_time,end_time,train_time,train_epoch,epoch_val_mIoU,avg_epoch_time
0,1,0,FALSE_random_x2_FALSE_none,jYQpo,2022-06-19 19:41:16,2022-06-19 19:41:26,2022-06-20 01:45:00,0 days 06:03:34,76,0.697371,0 days 00:04:47.026315789
1,1,0,FALSE_mskf_x2_FALSE_none,m7ltG,2022-06-20 01:45:00,2022-06-20 01:45:10,2022-06-20 08:07:31,0 days 06:22:21,80,0.709051,0 days 00:04:46.762500
2,1,0,TRUE_random_x2_FALSE_none,sKpWx,2022-06-20 08:07:31,2022-06-20 08:07:41,2022-06-20 12:26:32,0 days 04:18:51,79,0.708310,0 days 00:03:16.594936708


In [20]:
result_df = pd.merge(log_df, result_df, on= ["configs", "random_state"], how="left")



In [21]:
result_df.head()

,random_state,split_fnum,configs,job_name,config_time,start_time,end_time,train_time,train_epoch,epoch_val_mIoU,avg_epoch_time,is_imbalance,split_strategy,resampling_strategy,do_copypaste,paste_by,test_mIoU,aeroplane,bicycle,bird,boat,bottle,bus,car,cat,chair,cow,diningtable,dog,horse,motorbike,person,pottedplant,sheep,sofa,train,tvmonitor,background
0,1,0,FALSE_random_x2_FALSE_none,jYQpo,2022-06-19 19:41:16,2022-06-19 19:41:26,2022-06-20 01:45:00,0 days 06:03:34,76,0.697371,0 days 00:04:47.026315789,False,random,x2,False,none,0.557497,0.652341,0.290290,0.635696,0.520608,0.347514,0.735243,0.436864,0.759840,0.196924,0.736267,0.466755,0.696865,0.607881,0.662733,0.564343,0.220733,0.632440,0.450738,0.727686,0.486342,0.879340
1,1,0,FALSE_mskf_x2_FALSE_none,m7ltG,2022-06-20 01:45:00,2022-06-20 01:45:10,2022-06-20 08:07:31,0 days 06:22:21,80,0.709051,0 days 00:04:46.762500,False,mskf,x2,False,none,0.553489,0.649023,0.276470,0.653448,0.519311,0.347857,0.712158,0.492618,0.755708,0.168507,0.712359,0.452551,0.671878,0.631143,0.655712,0.569532,0.203210,0.658307,0.427302,0.728016,0.458932,0.879225
2,1,0,TRUE_random_x2_FALSE_none,sKpWx,2022-06-20 08:07:31,2022-06-20 08:07:41,2022-06-20 12:26:32,0 days 04:18:51,79,0.708310,0 days 00:03:16.594936708,True,random,x2,False,none,0.523494,0.569173,0.238537,0.541929,0.508524,0.264707,0.725048,0.477476,0.671550,0.042267,0.710797,0.383865,0.711472,0.625931,0.658192,0.552872,0.266131,0.592191,0.400734,0.707999,0.471167,0.872811
3,1,0,TRUE_mskf_cnt_FALSE_none,HYdBk,2022-06-20 12:26:33,2022-06-20 12:26:43,2022-06-20 17:02:05,0 days 04:35:22,84,0.725555,0 days 00:03:16.690476190,True,mskf,cnt,False,none,0.523744,0.597629,0.234189,0.526076,0.501576,0.285600,0.695156,0.446776,0.692041,0.053670,0.707520,0.410013,0.695629,0.633472,0.639259,0.544872,0.252745,0.615439,0.418436,0.731712,0.444342,0.872465
4,1,0,TRUE_skf_prop_cluster_FALSE_none,ZlfNW,2022-06-20 17:02:05,2022-06-20 17:02:15,2022-06-20 21:58:28,0 days 04:56:13,92,0.735685,0 days 00:03:13.184782608,True,skfprop,cluster,False,none,0.512570,0.552205,0.171125,0.488038,0.503097,0.284481,0.706901,0.416296,0.701284,0.044661,0.734790,0.328529,0.695073,0.635188,0.637767,0.555330,0.231980,0.601085,0.413620,0.712856,0.480403,0.869257


In [22]:
train_df = pd.read_csv('./data/train_df.csv', index_col=0)

In [23]:
pixel_cols = [i for i in train_df.columns if 'sum' in i]
cnt_cols = [i for i in train_df.columns if 'cnt' in i]
prop_cols = [i for i in train_df.columns if 'prop' in i]

In [24]:
split_strategies = ["random", "mskf", "skf_prop"]


In [25]:
data_stats = []

for s_st in split_strategies:
    for rs in range(1,6):
        cur_split = f"{s_st}_rstate{rs}_fold0"
        
        condition = ((train_df[cur_split] == 'train')&(train_df["use_image"]))
        
        num_train = len(train_df[condition])
        
        is_imbalance = True

        train_info = pd.Series(dict(cur_data=cur_split, num_train=num_train, is_imbalance=is_imbalance,
                                    split_strategy= s_st, random_state = rs))

        freq_sum = train_df[condition][cnt_cols].sum()
        
        proportion_mean = train_df[condition][prop_cols].mean()
        nz_proportion_mean = train_df[condition][prop_cols].replace(0,np.NaN).mean(numeric_only=True)
        
        pixel_mean = train_df[condition][pixel_cols].mean()
        nz_pixel_mean = train_df[condition][pixel_cols].replace(0,np.NaN).mean(numeric_only=True)
        
        nz_proportion_mean = nz_proportion_mean.rename(lambda x: x.replace('prop', 'nz_prop'))
        pixel_mean = pixel_mean.rename(lambda x: x.replace('sum', 'pix'))
        nz_pixel_mean = nz_pixel_mean.rename(lambda x: x.replace('sum', 'nz_pix'))

        cur_stats = pd.concat([train_info, freq_sum, pixel_mean, proportion_mean, nz_pixel_mean, nz_proportion_mean])
        data_stats.append(cur_stats)
        
        
split_strategies = ["random", "mskf"]
for s_st in split_strategies:
    for rs in range(1,6):
        cur_split = f"{s_st}_rstate{rs}_fold0"
        condition = ((train_df[cur_split] == 'train'))

        num_train = len(train_df[condition])
        
        is_imbalance = False
        train_info = pd.Series(dict(cur_data=cur_split, num_train=num_train, is_imbalance=is_imbalance,
                                    split_strategy= s_st, random_state = rs))

        freq_sum = train_df[condition][cnt_cols].sum()

        proportion_mean = train_df[condition][prop_cols].mean()
        nz_proportion_mean = train_df[condition][prop_cols].replace(0,np.NaN).mean(numeric_only=True)

        pixel_mean = train_df[condition][pixel_cols].mean()
        nz_pixel_mean = train_df[condition][pixel_cols].replace(0,np.NaN).mean(numeric_only=True)

        nz_proportion_mean = nz_proportion_mean.rename(lambda x: x.replace('prop', 'nz_prop'))
        pixel_mean = pixel_mean.rename(lambda x: x.replace('sum', 'pix'))
        nz_pixel_mean = nz_pixel_mean.rename(lambda x: x.replace('sum', 'nz_pix'))

        cur_stats = pd.concat([train_info, freq_sum, pixel_mean, proportion_mean, nz_pixel_mean, nz_proportion_mean])
        data_stats.append(cur_stats)

data_stats_df = pd.DataFrame(data_stats)

In [26]:
data_stats_df['random_state'] = data_stats_df.random_state.astype(str)

In [27]:
result_df = pd.merge(result_df, data_stats_df, on=['is_imbalance', 'split_strategy', 'random_state'], how='left')

In [28]:
result_df.tail(3)

,random_state,split_fnum,configs,job_name,config_time,start_time,end_time,train_time,train_epoch,epoch_val_mIoU,avg_epoch_time,is_imbalance,split_strategy,resampling_strategy,do_copypaste,paste_by,test_mIoU,aeroplane,bicycle,bird,boat,bottle,bus,car,cat,...,tvmonitor_nz_pix,background_nz_pix,border_nz_pix,aeroplane_nz_prop,bicycle_nz_prop,bird_nz_prop,boat_nz_prop,bottle_nz_prop,bus_nz_prop,car_nz_prop,cat_nz_prop,chair_nz_prop,cow_nz_prop,diningtable_nz_prop,dog_nz_prop,horse_nz_prop,motorbike_nz_prop,person_nz_prop,pottedplant_nz_prop,sheep_nz_prop,sofa_nz_prop,train_nz_prop,tvmonitor_nz_prop,background_nz_prop,border_nz_prop
47,3,0,TRUE_mskf_x2_FALSE_none,pQB4D,2022-07-08 00:24:53,2022-07-08 00:25:03,2022-07-08 04:45:07,0 days 04:20:04,82,0.715536,0 days 00:03:10.292682926,True,mskf,x2,False,none,0.517663,0.592730,0.239358,0.450943,0.523369,0.174862,0.733661,0.435887,0.682374,...,32206.829268,124930.483709,8979.309136,0.094242,0.090221,0.128528,0.121217,0.181127,0.321190,0.174920,0.311777,0.136042,0.178557,0.241865,0.205713,0.197014,0.210071,0.134524,0.145238,0.212287,0.244514,0.236979,0.174070,0.697831,0.050678
48,4,0,TRUE_mskf_x2_FALSE_none,bD8E1,2022-07-08 04:45:07,2022-07-08 04:45:17,2022-07-08 08:14:44,0 days 03:29:27,66,0.716589,0 days 00:03:10.409090909,True,mskf,x2,False,none,0.517067,0.527318,0.234094,0.509797,0.490629,0.283702,0.708980,0.465022,0.642727,...,32982.585366,124207.906015,8777.944862,0.130954,0.093415,0.087032,0.114267,0.183179,0.314265,0.162620,0.322873,0.129133,0.183687,0.217094,0.210723,0.192230,0.206336,0.146220,0.161420,0.213740,0.274586,0.262666,0.176385,0.694403,0.049559
49,5,0,TRUE_mskf_x2_FALSE_none,orS94,2022-07-08 08:14:44,2022-07-08 08:14:54,2022-07-08 11:50:06,0 days 03:35:12,68,0.722596,0 days 00:03:09.882352941,True,mskf,x2,False,none,0.524217,0.590151,0.237540,0.529795,0.512695,0.284224,0.724877,0.433761,0.695221,...,26282.975610,125082.895597,8696.346734,0.115723,0.097352,0.127703,0.114715,0.107090,0.316531,0.157806,0.258600,0.134952,0.172318,0.243299,0.214182,0.190100,0.203209,0.137254,0.152762,0.180393,0.255663,0.266767,0.141175,0.701878,0.049107


In [29]:
result_df.to_csv('./data/final_result.csv', index=False)

In [30]:
result_df

,random_state,split_fnum,configs,job_name,config_time,start_time,end_time,train_time,train_epoch,epoch_val_mIoU,avg_epoch_time,is_imbalance,split_strategy,resampling_strategy,do_copypaste,paste_by,test_mIoU,aeroplane,bicycle,bird,boat,bottle,bus,car,cat,...,tvmonitor_nz_pix,background_nz_pix,border_nz_pix,aeroplane_nz_prop,bicycle_nz_prop,bird_nz_prop,boat_nz_prop,bottle_nz_prop,bus_nz_prop,car_nz_prop,cat_nz_prop,chair_nz_prop,cow_nz_prop,diningtable_nz_prop,dog_nz_prop,horse_nz_prop,motorbike_nz_prop,person_nz_prop,pottedplant_nz_prop,sheep_nz_prop,sofa_nz_prop,train_nz_prop,tvmonitor_nz_prop,background_nz_prop,border_nz_prop
0,1,0,FALSE_random_x2_FALSE_none,jYQpo,2022-06-19 19:41:16,2022-06-19 19:41:26,2022-06-20 01:45:00,0 days 06:03:34,76,0.697371,0 days 00:04:47.026315789,False,random,x2,False,none,0.557497,0.652341,0.290290,0.635696,0.520608,0.347514,0.735243,0.436864,0.759840,...,29494.088235,124135.602740,9889.766866,0.120479,0.061706,0.117315,0.118679,0.096262,0.301848,0.157293,0.292609,0.112151,0.189372,0.230505,0.198306,0.194391,0.197004,0.155646,0.108732,0.197973,0.228286,0.268418,0.157874,0.691476,0.055353
1,1,0,FALSE_mskf_x2_FALSE_none,m7ltG,2022-06-20 01:45:00,2022-06-20 01:45:10,2022-06-20 08:07:31,0 days 06:22:21,80,0.709051,0 days 00:04:46.762500,False,mskf,x2,False,none,0.553489,0.649023,0.276470,0.653448,0.519311,0.347857,0.712158,0.492618,0.755708,...,29911.417910,124947.130993,10018.830914,0.120918,0.055538,0.114568,0.106267,0.091414,0.305744,0.155082,0.287179,0.113857,0.188486,0.241925,0.200665,0.200496,0.202002,0.150128,0.101752,0.190774,0.197239,0.264160,0.160539,0.694315,0.055992
2,1,0,TRUE_random_x2_FALSE_none,sKpWx,2022-06-20 08:07:31,2022-06-20 08:07:41,2022-06-20 12:26:32,0 days 04:18:51,79,0.708310,0 days 00:03:16.594936708,True,random,x2,False,none,0.523494,0.569173,0.238537,0.541929,0.508524,0.264707,0.725048,0.477476,0.671550,...,31235.136364,125013.505618,8647.516854,0.093482,0.085143,0.126022,0.121408,0.194825,0.307243,0.168325,0.292201,0.155681,0.185155,0.215470,0.207301,0.194250,0.199626,0.144454,0.139317,0.196613,0.262362,0.268418,0.167178,0.696699,0.048640
3,1,0,TRUE_mskf_cnt_FALSE_none,HYdBk,2022-06-20 12:26:33,2022-06-20 12:26:43,2022-06-20 17:02:05,0 days 04:35:22,84,0.725555,0 days 00:03:16.690476190,True,mskf,cnt,False,none,0.523744,0.597629,0.234189,0.526076,0.501576,0.285600,0.695156,0.446776,0.692041,...,33115.950000,125737.558198,8874.857500,0.129080,0.056959,0.092099,0.106941,0.156390,0.311269,0.164329,0.291159,0.152472,0.184169,0.253758,0.210619,0.200463,0.204906,0.140523,0.131151,0.191527,0.234703,0.264160,0.177489,0.700213,0.049825
4,1,0,TRUE_skf_prop_cluster_FALSE_none,ZlfNW,2022-06-20 17:02:05,2022-06-20 17:02:15,2022-06-20 21:58:28,0 days 04:56:13,92,0.735685,0 days 00:03:13.184782608,True,skfprop,cluster,False,none,0.512570,0.552205,0.171125,0.488038,0.503097,0.284481,0.706901,0.416296,0.701284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,0,TRUE_random_x2_TRUE_cnt,Iufqb,2022-06-20 21:58:28,2022-06-20 21:58:38,2022-06-21 02:08:25,0 days 04:09:47,67,0.707435,0 days 00:03:43.686567164,True,random,x2,True,cnt,0.529637,0.599052,0.257521,0.552390,0.491578,0.347143,0.720674,0.448277,0.659682,...,31235.136364,125013.505618,8647.516854,0.093482,0.085143,0.126022,0.121408,0.194825,0.307243,0.168325,0.292201,0.155681,0.185155,0.215470,0.207301,0.194250,0.199626,0.144454,0.139317,0.196613,0.262362,0.268418,0.167178,0.696699,0.048640
6,1,0,TRUE_random_x2_TRUE_performance,soH9t,2022-06-21 02:08:25,2022-06-21 02:08:35,2022-06-21 06:26:02,0 days 04:17:27,69,0.705088,0 days 00:03:43.869565217,True,random,x2,True,performance,0.530236,0.529957,0.252760,0.548708,0.534469,0.319646,0.721082,0.470684,0.641883,...,31235.136364,125013.505618,8647.516854,0.093482,0.085143,0.126022,0.121408,0.194825,0.307243,0.168325,0.292201,0.155681,0.185155,0.215470,0.207301,0.194250,0.199626,0.144454,0.139317,0.196613,0.262362,0.268418,0.167178,0.